In [244]:
import requests
import pandas as pd
import numpy as np
from datetime import datetime
from bs4 import BeautifulSoup
import json

In [245]:
df = pd.read_csv('database.csv')

In [246]:
df['competences_matched'] = df['competences_matched'].str.replace('[', '').str.replace(']', '')

In [247]:
df['competences_matched'] = df['competences_matched'].str.split(',').apply(lambda x: [item.strip("' ") for item in x])
competences_list = df['competences_matched'].explode().tolist()


In [248]:
df['competences_matched'] = df['competences_matched'].apply(lambda x : set(x))
df['competences_matched'] = df['competences_matched'].apply(lambda x : list(x))

In [249]:
df_machine_learning = df.explode('competences_matched')

In [250]:
df_machine_learning.iloc[:, 14:].columns

Index([], dtype='object')

In [251]:
# One-hot encode the 'competences_matched' column
competences_dummies = df_machine_learning['competences_matched'].str.get_dummies(sep=', ')

# Concatenate the one-hot encoded competences with the original dataframe
df_machine_learning = pd.concat([df_machine_learning, competences_dummies], axis=1)

In [252]:
columns_to_sum = ['agile', 'angular', 'apis', 'aws', 'azure', 'backend', 'bash', 'devops', 'docker', 'etl', 'excel', 'frontend', 'git', 'github', 'gitlab', 'hadoop', 'html', 'java', 'javascript', 'kanban', 'linux', 'matplotlib', 'mongodb', 'mysql', 'numpy', 'postgresql', 'python', 'r', 'react', 'scrum', 'seaborn', 'spark', 'sql', 'sqlite', 'tableau', 'unix']

result = df_machine_learning.groupby('id')[columns_to_sum].sum().reset_index()
result

,id,agile,angular,apis,aws,azure,backend,bash,devops,docker,...,python,r,react,scrum,seaborn,spark,sql,sqlite,tableau,unix
0,0054659,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0123421,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,1,0,0,0,0
2,0147119,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0157648,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0221865,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,req4757,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
495,req4766,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
496,req7605,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,0
497,v2q77mpg8aJGyZpr,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [253]:
df_ML = df.merge(result, how='inner', left_on='id', right_on='id')

In [254]:
df_ML.columns

Index(['id', 'offre', 'url_page', 'ville', 'pays', 'entreprise', 'contrat',
       'date_publi', 'id_plateforme', 'description', 'lat', 'lon',
       'competences', 'competences_matched', 'agile', 'angular', 'apis', 'aws',
       'azure', 'backend', 'bash', 'devops', 'docker', 'etl', 'excel',
       'frontend', 'git', 'github', 'gitlab', 'hadoop', 'html', 'java',
       'javascript', 'kanban', 'linux', 'matplotlib', 'mongodb', 'mysql',
       'numpy', 'postgresql', 'python', 'r', 'react', 'scrum', 'seaborn',
       'spark', 'sql', 'sqlite', 'tableau', 'unix'],
      dtype='object')

In [255]:
col_drop = ['pays', 'description', 'url_page', 'id', 'competences', 'competences_matched']

In [256]:
df_ML = df_ML.drop(columns=col_drop)

In [257]:
df_ML = df_ML[df_ML['entreprise'].notna()]

In [258]:
df_ML = df_ML[df_ML['lat'].notna()]

In [259]:
df_choix = df_ML[:26]

In [260]:
%pip install -U scikit-learn

In [261]:
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import NearestNeighbors

X = df_ML

selector_num_cols = selector(dtype_exclude=object)
selector_cat_cols = selector(dtype_include=object)

num_cols = selector_num_cols(X)
cat_cols = selector_cat_cols(X)

num_preprocessor = StandardScaler()
cat_preprocessor = OneHotEncoder()
preprocessor = ColumnTransformer(
            [
                ("OneHotEncoder", cat_preprocessor, cat_cols),
                ("StandardScaler", num_preprocessor, num_cols),
            ]
        )
neigh = NearestNeighbors(n_neighbors=5)
X_scaled = preprocessor.fit_transform(X)
neigh.fit(X_scaled)
cible = df_choix
    
    
user = preprocessor.transform(cible)
    
array = neigh.kneighbors(user)
array

(array([[0.        , 2.75474688, 2.82842778, 2.82865887, 2.84006023],
        [0.        , 5.11096937, 5.98614128, 6.11854376, 6.97966505],
        [0.        , 2.82842712, 3.14226157, 3.28230227, 3.28964897],
        [0.        , 1.41421356, 2.82842712, 2.82844567, 2.82852355],
        [0.        , 2.96845851, 3.11488477, 3.16657301, 3.2172247 ],
        [0.        , 1.41421356, 2.82842712, 2.82844567, 2.82852355],
        [0.        , 2.8636164 , 2.96066724, 3.07851005, 3.08210084],
        [0.        , 3.1623263 , 3.17630284, 3.45946323, 3.45973486],
        [0.        , 2.46116191, 2.74035829, 4.40329732, 7.19525828],
        [0.        , 2.44949148, 3.78641083, 3.88563626, 3.95140758],
        [0.        , 5.02858951, 5.60045042, 5.77624834, 6.51257999],
        [0.        , 2.82842712, 3.12139056, 3.12139056, 3.12139056],
        [0.        , 2.82842712, 3.14226157, 3.15636492, 3.42136209],
        [0.        , 2.82867622, 2.94653412, 2.99688194, 3.4835094 ],
        [0.        ,

In [262]:
list_chiffre

[0.0, 0.0, 0.0, 0.0, 0.0]

In [263]:
array[0]

array([[0.        , 2.75474688, 2.82842778, 2.82865887, 2.84006023],
       [0.        , 5.11096937, 5.98614128, 6.11854376, 6.97966505],
       [0.        , 2.82842712, 3.14226157, 3.28230227, 3.28964897],
       [0.        , 1.41421356, 2.82842712, 2.82844567, 2.82852355],
       [0.        , 2.96845851, 3.11488477, 3.16657301, 3.2172247 ],
       [0.        , 1.41421356, 2.82842712, 2.82844567, 2.82852355],
       [0.        , 2.8636164 , 2.96066724, 3.07851005, 3.08210084],
       [0.        , 3.1623263 , 3.17630284, 3.45946323, 3.45973486],
       [0.        , 2.46116191, 2.74035829, 4.40329732, 7.19525828],
       [0.        , 2.44949148, 3.78641083, 3.88563626, 3.95140758],
       [0.        , 5.02858951, 5.60045042, 5.77624834, 6.51257999],
       [0.        , 2.82842712, 3.12139056, 3.12139056, 3.12139056],
       [0.        , 2.82842712, 3.14226157, 3.15636492, 3.42136209],
       [0.        , 2.82867622, 2.94653412, 2.99688194, 3.4835094 ],
       [0.        , 2.17900233, 2.

In [267]:
len(array[0])

26

In [268]:
array[0]

array([[0.        , 2.75474688, 2.82842778, 2.82865887, 2.84006023],
       [0.        , 5.11096937, 5.98614128, 6.11854376, 6.97966505],
       [0.        , 2.82842712, 3.14226157, 3.28230227, 3.28964897],
       [0.        , 1.41421356, 2.82842712, 2.82844567, 2.82852355],
       [0.        , 2.96845851, 3.11488477, 3.16657301, 3.2172247 ],
       [0.        , 1.41421356, 2.82842712, 2.82844567, 2.82852355],
       [0.        , 2.8636164 , 2.96066724, 3.07851005, 3.08210084],
       [0.        , 3.1623263 , 3.17630284, 3.45946323, 3.45973486],
       [0.        , 2.46116191, 2.74035829, 4.40329732, 7.19525828],
       [0.        , 2.44949148, 3.78641083, 3.88563626, 3.95140758],
       [0.        , 5.02858951, 5.60045042, 5.77624834, 6.51257999],
       [0.        , 2.82842712, 3.12139056, 3.12139056, 3.12139056],
       [0.        , 2.82842712, 3.14226157, 3.15636492, 3.42136209],
       [0.        , 2.82867622, 2.94653412, 2.99688194, 3.4835094 ],
       [0.        , 2.17900233, 2.

In [280]:
list_chiffre = []
index = []

for i in array[0] :
    for y in i :
        # print(y)
        list_chiffre.append(y)
list_chiffre.sort()
list_chiffre = list_chiffre[len(array[0]):]

In [284]:
for i in range(len(list_chiffre[:5])):
    # display(df_ML.iloc[array[1][0][i]])
    print(df_ML.iloc[array[1][0][i]].name)
    index.append(df_ML.iloc[array[1][0][i]].name)

0
17
354
191
460


In [285]:
df_ML[df_ML.index.isin(index)]

,offre,ville,entreprise,contrat,date_publi,id_plateforme,lat,lon,agile,angular,...,python,r,react,scrum,seaborn,spark,sql,sqlite,tableau,unix
0,Business 1 Data Analyst H/F,Île-de-France,Eni Gas & Power France,CDI,2024-02-08,Hello Work,48.856600,2.352200,0,0,...,0,0,0,0,0,0,0,0,1,0
17,Analyste Modèles & Data H/F,Île-de-France,BPCE SA,CDI,2024-02-08,Hello Work,48.856600,2.352200,0,0,...,0,0,0,0,0,0,0,0,0,0
191,Data Analyste H/F,Saint-Cloud,Elis,CDI,2024-02-01,Hello Work,48.861274,2.212124,0,0,...,0,0,0,0,0,0,0,0,1,0
354,Data Analyst H/F,Paris,Mydral,CDI,2024-01-23,Hello Work,48.869478,2.346192,0,0,...,0,0,0,0,0,0,0,0,1,0
460,Data Analyst H/F,Villeneuve-d'Ascq,Cofidis,CDI,2024-01-11,Hello Work,50.605471,3.149605,0,0,...,0,0,0,0,0,0,0,0,1,0


In [ ]:
# python -m streamlit run c:/Users/theov/Desktop/PROJET/PROJET_3/app.py

In [ ]:
df_user = pd.read_csv('database_user.csv')